# Graph Neural Networks with Pytorch

Original Paper: https://arxiv.org/abs/1706.02216    
Original Code: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/reddit.py

In [ ]:
!pip install pytorch torchvision torchaudio
!pip install torch_scatter
!pip install torch_sparse

In [1]:
import os
import sys

import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.datasets import Reddit
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv
from torch.optim.lr_scheduler import StepLR


sys.path.append('./')
from utils import *
from model import *
logger = make_logger(name='graphsage_logger')

c:\Users\wikid\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = os.path.join(os.getcwd(), '..', 'data', 'Reddit')
dataset = Reddit(path)
data = dataset[0]

In [10]:
train_loader_size = [25, 10]
subgraph_loader_size = [-1]
num_worker = 8
epoch_num = 4
batch = 1024

In [4]:
logger.info(f"Node Feature Matrix Info: # Nodes: {data.x.shape[0]}, # Node Features: {data.x.shape[1]}")

logger.info(f"Edge Index Shape: {data.edge_index.shape}")
logger.info(f"Edge Weight: {data.edge_attr}")

print(data.train_mask.sum().item())

2023-03-09 12:27:12,278 - graphsage_logger - Node Feature Matrix Info: # Nodes: 232965, # Node Features: 602
2023-03-09 12:27:12,280 - graphsage_logger - Edge Index Shape: torch.Size([2, 114615892])
2023-03-09 12:27:12,281 - graphsage_logger - Edge Weight: None


153431


In [5]:
train_loader = NeighborSampler(
    data.edge_index, node_idx=data.train_mask,
    sizes=train_loader_size, batch_size=batch, shuffle=True, num_workers=num_worker)

subgraph_loader = NeighborSampler(
    data.edge_index, node_idx=None,
    sizes=subgraph_loader_size, batch_size=batch, shuffle=False, num_workers=num_worker)

c:\Users\wikid\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.NeighborSampler' is deprecated, use 'loader.NeighborSampler' instead
  warnings.warn(out)


In [6]:
batch_size, n_id, adjs = next(iter(train_loader))

logger.info(f"Current Batch Size: {batch_size}")
logger.info(f"현재 Subgraph에서 사용된 모든 node id의 개수: {n_id.shape[0]}")
logger.info(f"Layer의 수: {len(adjs)}")

A = adjs[1].size[0] - batch_size
B = adjs[0].size[0] - A - batch_size

logger.info(f"진행 방향: {B}개의 2-hop neighbors ->"
            f"{A}개의 1-hop neighbors -> {batch_size}개의 Head Nodes")

2023-03-09 12:28:34,941 - graphsage_logger - Current Batch Size: 1024
2023-03-09 12:28:34,943 - graphsage_logger - 현재 Subgraph에서 사용된 모든 node id의 개수: 106067
2023-03-09 12:28:34,945 - graphsage_logger - Layer의 수: 2
2023-03-09 12:28:34,946 - graphsage_logger - 진행 방향: 84345개의 2-hop neighbors ->20698개의 1-hop neighbors -> 1024개의 Head Nodes


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGE(dataset.num_features, 256, dataset.num_classes)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

x = data.x.to(device)
y = data.y.squeeze().to(device)

In [8]:
def train(epoch):
    model.train()

    pbar = tqdm(total=int(data.train_mask.sum()))
    pbar.set_description(f'Epoch {epoch:02d}')

    total_loss = total_correct = 0
    for batch_size, n_id, adjs in train_loader:
        adjs = [adj.to(device) for adj in adjs]

        optimizer.zero_grad()
        out = model(x[n_id], adjs)
        loss = F.nll_loss(out, y[n_id[:batch_size]])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())
        pbar.update(batch_size)

    pbar.close()

    loss = total_loss / len(train_loader)
    approx_acc = total_correct / int(data.train_mask.sum())
    return loss, approx_acc

@torch.no_grad()
def test():
    model.eval()
    out = model.inference(x, subgraph_loader, device)

    y_true = y.cpu().unsqueeze(-1)
    y_pred = out.argmax(dim=-1, keepdim=True)

    results = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        results += [int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]
    return results

In [2]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, save_path='checkpoint.pt'):
        """
        :param patience: how many times you will wait before earlystopping
        :param save_path: where to save checkpoint
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.save_path = save_path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model, val_loss)
        elif score < self.best_score:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model, val_loss)
            self.counter = 0 # reset

    def save_checkpoint(self, model, val_loss):
        if self.verbose:
            print(f"val loss: ({self.val_loss_min:.6f} -> {val_loss:.6f})")
        torch.save(model.state_dict(), self.save_path)
        self.val_loss_min = val_loss

In [11]:
for epoch in range(1, epoch_num):
    loss, acc = train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')
    scheduler.step()
    train_acc, val_acc, test_acc = test()
    print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Epoch 01: 100%|██████████| 153431/153431 [02:57<00:00, 862.34it/s] 


Epoch 01, Loss: 0.4545, Approx. Train: 0.9227


Evaluating: 100%|██████████| 465930/465930 [07:17<00:00, 1065.67it/s]


Train: 0.9522, Val: 0.9507, Test: 0.9483


Epoch 02: 100%|██████████| 153431/153431 [03:09<00:00, 808.44it/s] 


...

Train: 0.9538, Val: 0.9499, Test: 0.9484


Epoch 03: 100%|██████████| 153431/153431 [03:31<00:00, 726.52it/s]


Epoch 03, Loss: 0.5822, Approx. Train: 0.9247


Evaluating: 100%|██████████| 465930/465930 [08:08<00:00, 954.74it/s] 

Train: 0.9529, Val: 0.9469, Test: 0.9463
